# Assignment 5



In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [2]:
!pip install selenium
!pip install webdriver-manager

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
import re
import os
import time 
import pandas as pd

In [4]:
# Setting up Chrome options for WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("executable_path=C:\\Users\\HP\\Documents\\tareas\\chromedriver-win64\\chromedriver.exe")

# Initializing the WebDriver with configured options
driver = webdriver.Chrome(options=chrome_options)

# Maximizing the browser window and zoom
driver.maximize_window()
driver.execute_script("document.body.style.zoom='100%'")

# Base URL for the web scraping
url_base = "https://infogob.jne.gob.pe/Eleccion"

# Lists to store Election Years and DataFrames
eleccion_year_list = []
dataframes_list = []

# Looping through election years. LISTO
for div in range(2, 27):
    
    # Opening the base URL in the WebDriver
    driver.get(url_base)
        # Interacting with dropdowns and elements to navigate to the desired election
    dropdown_paths_1 = [
        '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div',
        '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]',
        '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div'
         ]
    for path_1 in dropdown_paths_1:
        driver.find_element(By.XPATH, path_1).click()
        time.sleep(1)

    # Constructing XPath to locate the specific election element based on the loop variable 'div'
    # The format function inserts the current value of 'div' into the XPath
    election_xpath = '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(div)    
    driver.find_element(By.XPATH, election_xpath).click()

    # Getting the election year and adding it to the list
    eleccion_year = driver.find_element(By.XPATH, election_xpath).text
    eleccion_year_list.append(eleccion_year)

    time.sleep(1)
    
    # Interacting with 'Ver datos de la elección' and 'Candidatos y resultados'
    dropdown_paths_2 = [
        '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button',
        '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a'
    ]
    for path_2 in dropdown_paths_2:
        
        driver.find_element(By.XPATH, path_2).click()
        time.sleep(2)
    
    # Getting the HTML of the election results table
    table_path = driver.find_element(By.ID,'gridEleccionResultadosElectorales')
    table_html = table_path.get_attribute( 'innerHTML' )
    
    # Getting the table directly using Pandas
    df = pd.read_html(table_path.get_attribute('outerHTML'))[0]

    # We create a new DataFrame and insert the list that we obtained previously 
    # We place it at the beginning [0] and with a new name    
    df_cleaned = df.iloc[:, :].copy()
    df_cleaned.insert(0, 'Elecciones', eleccion_year)

    # Storing the DataFrame in the list
    dataframes_list.append(df_cleaned)
    
# Concatenating all the DataFrames into one, removing the columns we don't want
final_df = pd.concat(dataframes_list, ignore_index=True)
columns_to_drop = ['SÍMBOLO', 'PLAN DE GOBIERNO', 'PORCENTAJE DE VOTOS VÁLIDOS', 'LISTA DE CANDIDATOS']
final_df = final_df.drop(columns=columns_to_drop, errors='ignore')

# Quitting the WebDriver at the end
driver.quit()

# Displaying the final DataFrame
final_df

# Saving the DataFrame to an Excel file
excel_file_path = r'C:\Users\HP\Documents\GitHub\QLAB_Summer_Python\assignments\assignment_5\group_6_ass_5_2024.xlsx'
final_df.to_excel(excel_file_path, index=False)
